In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

tff.backends.reference.set_reference_context()

In [3]:
import sys
sys.path.append("../")
from federated.data.mitbih_data_preprocessing import load_data, create_unbalanced_data

In [4]:
NUMBER_OF_CLIENTS=5
CLIENT_EPOCHS=10
BATCH_SIZE=32
NUM_EXAMPLES_PER_USER = 20_000

In [5]:
train_client_data, test_client_data = load_data(normalized=True, data_selector=create_unbalanced_data)

In [6]:
def get_data_for_ecg(source, client):
    output_sequence = []
    data = source[f"client_{client}"]
    all_samples = [i for i, y in enumerate(data[1])]
    for i in range(0, min(len(all_samples), NUM_EXAMPLES_PER_USER), BATCH_SIZE):
        batch_samples = all_samples[i:i + BATCH_SIZE]
        output_sequence.append({
            "x": np.array([data[0][i] for i in batch_samples], dtype=np.float32),
            "y": np.array([np.where(data[1][i]==1)[0][0] for i in batch_samples], dtype=np.int32)
        })
    return output_sequence

federated_train_data = [get_data_for_ecg(train_client_data, client) for client in range(1, 6)]

federated_test_data = [get_data_for_ecg(train_client_data, client) for client in range(1, 6)]

In [7]:
train_client_data["client_3"]

 array([0, 1, 0, 0, 0]),
  array([0, 1, 0, 0, 0]),
  array([0, 0, 0, 1, 0]),
  array([0, 0, 0, 0, 1]),
  array([0, 0, 0, 1, 0]),
  array([0, 0, 1, 0, 0]),
  array([0, 0, 1, 0, 0]),
  array([0, 1, 0, 0, 0]),
  array([0, 0, 0, 1, 0]),
  array([0, 0, 0, 0, 1]),
  array([0, 0, 0, 0, 1]),
  array([0, 1, 0, 0, 0]),
  array([0, 0, 1, 0, 0]),
  array([0, 1, 0, 0, 0]),
  array([0, 0, 1, 0, 0]),
  array([0, 0, 0, 1, 0]),
  array([0, 0, 0, 1, 0]),
  array([0, 0, 0, 1, 0]),
  array([0, 0, 0, 1, 0]),
  array([0, 0, 0, 1, 0]),
  array([0, 0, 0, 1, 0]),
  array([0, 1, 0, 0, 0]),
  array([0, 1, 0, 0, 0]),
  array([0, 0, 1, 0, 0]),
  array([0, 0, 0, 1, 0]),
  array([0, 1, 0, 0, 0]),
  array([0, 1, 0, 0, 0]),
  array([0, 0, 0, 1, 0]),
  array([0, 0, 0, 0, 1]),
  array([0, 0, 1, 0, 0]),
  array([0, 1, 0, 0, 0]),
  array([0, 0, 0, 0, 1]),
  array([0, 0, 0, 0, 1]),
  array([0, 0, 0, 0, 1]),
  array([0, 0, 1, 0, 0]),
  array([0, 1, 0, 0, 0]),
  array([0, 1, 0, 0, 0]),
  array([0, 0, 1, 0, 0]),
  array([0, 1

In [8]:
BATCH_SPEC = collections.OrderedDict(
    x=tf.TensorSpec(shape=[None, 186], dtype=tf.float32),
    y=tf.TensorSpec(shape=[None], dtype=tf.int32))
BATCH_TYPE = tff.to_type(BATCH_SPEC)

str(BATCH_TYPE)

'<x=float32[?,186],y=int32[?]>'

In [9]:
MODEL_SPEC = collections.OrderedDict(
    weights=tf.TensorSpec(shape=[186, 5], dtype=tf.float32),
    bias=tf.TensorSpec(shape=[5], dtype=tf.float32))
MODEL_TYPE = tff.to_type(MODEL_SPEC)

print(MODEL_TYPE)

<weights=float32[186,5],bias=float32[5]>


In [10]:
@tf.function
def forward_pass(model, batch):
    predicted_y = tf.nn.softmax(
        tf.matmul(batch['x'], model['weights']) + model['bias'])
    return -tf.reduce_mean(
        tf.reduce_sum(
            tf.one_hot(batch['y'], 5) * tf.math.log(predicted_y), axis=[1]))

@tff.tf_computation(MODEL_TYPE, BATCH_TYPE)
def batch_loss(model, batch):
    return forward_pass(model, batch)

In [11]:
initial_model = collections.OrderedDict(
    weights=np.zeros([186, 5], dtype=np.float32),
    bias=np.zeros([5], dtype=np.float32))

sample_batch = federated_train_data[4][-1]

batch_loss(initial_model, sample_batch)

1.609438

In [12]:
@tff.tf_computation(MODEL_TYPE, BATCH_TYPE, tf.float32)
def batch_train(initial_model, batch, learning_rate):
    # Define a group of model variables and set them to `initial_model`. Must
    # be defined outside the @tf.function
    model_vars = collections.OrderedDict([
        (name, tf.Variable(name=name, initial_value=value))
        for name, value in initial_model.items()
    ])
    optimizer = tf.keras.optimizers.SGD(learning_rate)

    @tf.function
    def _train_on_batch(model_vars, batch):
        # Perform one step gradient descent using loss from `batch_loss`.
        with tf.GradientTape() as tape:
            loss = forward_pass(model_vars, batch)
        grads = tape.gradient(loss, model_vars)
        optimizer.apply_gradients(
            zip(tf.nest.flatten(grads), tf.nest.flatten(model_vars)))
        return model_vars

    return _train_on_batch(model_vars, batch)

In [13]:
LOCAL_DATA_TYPE = tff.SequenceType(BATCH_TYPE)

@tff.federated_computation(MODEL_TYPE, tf.float32, LOCAL_DATA_TYPE)
def local_train(initial_model, learning_rate, all_batches):

    # Mapping function to apply to each batch
    @tff.federated_computation(MODEL_TYPE, BATCH_TYPE)
    def batch_fn(model, batch):
        return batch_train(model, batch, learning_rate)
    
    return tff.sequence_reduce(all_batches, initial_model, batch_fn)

In [14]:
locally_trained_model = local_train(initial_model, 0.1, federated_train_data[4])

In [15]:
@tff.federated_computation(MODEL_TYPE, LOCAL_DATA_TYPE)
def local_eval(model, all_batches):
    return tff.sequence_sum(
        tff.sequence_map(
            tff.federated_computation(lambda b: batch_loss(model, b), BATCH_TYPE),
            all_batches))

In [16]:
print('initial_model loss =', local_eval(initial_model,
                                          federated_train_data[4]))

print('locally_trained_model loss =',
      local_eval(locally_trained_model, federated_train_data[4]))

initial_model loss = 1005.8976
locally_trained_model loss = 330.406


In [17]:
print('initial_model loss =', local_eval(initial_model,
                                          federated_train_data[0]))

print('locally_trained_model loss =',
      local_eval(locally_trained_model, federated_train_data[0]))

initial_model loss = 1005.8976
locally_trained_model loss = 4141.384


In [18]:
SERVER_MODEL_TYPE = tff.type_at_server(MODEL_TYPE)
CLIENT_DATA_TYPE = tff.type_at_clients(LOCAL_DATA_TYPE)

In [19]:
@tff.federated_computation(SERVER_MODEL_TYPE, CLIENT_DATA_TYPE)
def federated_eval(model, data):
    return tff.federated_mean(
        tff.federated_map(local_eval, [tff.federated_broadcast(model), data]))

In [20]:
print('initial_model loss =', federated_eval(initial_model,
                                          federated_train_data))

print('locally_trained_model loss =',
      federated_eval(locally_trained_model, federated_train_data))

initial_model loss = 1005.2538
locally_trained_model loss = 1089.9082


In [21]:
SERVER_FLOAT_TYPE = tff.type_at_server(tf.float32)

@tff.federated_computation(
    SERVER_MODEL_TYPE, SERVER_FLOAT_TYPE, CLIENT_DATA_TYPE)
def grab_all_models(model, learning_rate, data):
  return tff.federated_collect(
      tff.federated_map(
          local_train,
          [tff.federated_broadcast(model),
           tff.federated_broadcast(learning_rate),
           data]))

@tff.federated_computation(SERVER_MODEL_TYPE, SERVER_FLOAT_TYPE,
                           CLIENT_DATA_TYPE)
def federated_train(model, learning_rate, data):
    return tff.federated_mean(
        tff.federated_map(local_train, [
            tff.federated_broadcast(model),
            tff.federated_broadcast(learning_rate), data
        ]))

In [22]:
model = initial_model
learning_rate = 0.1
np.set_printoptions(threshold=sys.maxsize)
for round_num in range(6):
    model = federated_train(model, learning_rate, federated_train_data)
    learning_rate = learning_rate * 0.9
    loss = federated_eval(model, federated_train_data)
    print(f"round {round_num}, loss={loss}")

round 0, loss=753.0848999023438
round 1, loss=705.829345703125
round 2, loss=674.6227416992188
round 3, loss=653.8548583984375
round 4, loss=638.3298950195312
round 5, loss=625.63818359375


In [23]:
print('initial_model test loss =',
      federated_eval(initial_model, federated_test_data))
print('trained_model test loss =', federated_eval(model, federated_test_data))

initial_model test loss = 1005.2538
trained_model test loss = 625.6382


In [24]:
models = grab_all_models(model, 0.1, federated_train_data)

In [25]:
print(models[0])

OrderedDict([('weights', array([[ 1.20760393e+00,  1.05221939e+00, -2.25933433e+00,
        -3.94436777e-01,  3.93946826e-01],
       [ 9.20916080e-01,  7.48489738e-01, -2.19370782e-01,
         1.24408215e-01, -1.57444334e+00],
       [ 2.30448410e-01, -2.58774567e+00,  1.58416176e+00,
         6.14343047e-01,  1.58793673e-01],
       [-1.13559328e-02, -2.26310205e+00,  1.06009150e+00,
         2.04031691e-01,  1.01033318e+00],
       [ 5.93251139e-02, -4.96395886e-01,  4.97378297e-02,
        -5.21330655e-01,  9.08662975e-01],
       [ 8.97566825e-02,  5.33083361e-03, -1.80368990e-01,
        -5.07333994e-01,  5.92616618e-01],
       [ 7.39227533e-02, -1.19812727e-01,  1.48790151e-01,
        -4.18378919e-01,  3.15479487e-01],
       [ 7.64050409e-02,  1.15874074e-01,  3.34409595e-01,
        -4.57007557e-01, -6.96802214e-02],
       [ 8.68214965e-02,  3.20579201e-01,  3.22465211e-01,
        -3.12867016e-01, -4.16999280e-01],
       [ 1.12144813e-01,  2.69501805e-01,  3.10081601e-01

In [44]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(5, 5, figsize=(45,45))

for i in range(5):
    model = models[i]
    for j in range(5):
        axs[i, j].set_title(f"Model {i}, client {j}")
        w = model["weights"][:, j]
        axs[i, j].plot(w)
plt.show()

TypeError: __init__() got an unexpected keyword argument 'title'